In [5]:
# pacotes
import numpy as np
import pandas as pd

In [6]:
# oculta mensagens de avisos
import warnings
warnings.filterwarnings("ignore")

In [7]:
# pd.set_option('display.max_columns', None)

In [8]:
# pd.set_option('display.max_rows', None)

In [9]:
day = '20050103'

In [11]:
# factors dataframe
factors_path = f'../input/factors/{day}.csv'
factors = pd.read_csv(factors_path, index_col=0)
# precisamos dropar todas ações que não tem TAQ_TICKER, pois é a única variável q conseguimos ligar na outra base
factors = factors[factors['TAQ_TICKER'] != '<undefined>']
# precisamos arrumar os valores das colunas de market cap (torná-los todos positivos)
factors['MARKETCAP'] = factors['MARKETCAP'].abs()

In [12]:
# returns dataframe
returns_path = f'../input/returns/{day}.csv'
returns = pd.read_csv(returns_path, index_col=0)

In [13]:
col = 'pNYSE_size'

In [14]:
long_position = []      # posição comprado
short_position = []     # posição vendido  

# primeiro, precisamos dropar as linhas tais que tem valor NaN no parâmetro de localidade do fator
temp = factors[factors[col].notna()]
"""
Criamos um loop com o critério:
tercil de cima: '7' estritamente ('8' fracamente), então o portfolio assume a posição comprado
tercil de baixo: '4' estritamente ('3' fracamente), então o portfolio assume a posição vendido
"""
for permno in temp.index:
    if temp.loc[permno][col] >= 8:
        long_position.append(temp.loc[permno]['TAQ_TICKER'])
    elif temp.loc[permno][col] <= 3:
        short_position.append(temp.loc[permno]['TAQ_TICKER'])

In [16]:
drop_long_position = []
drop_short_position = []

for ticker in long_position:
    if ticker not in returns.columns:
        drop_long_position.append(ticker)

for ticker in drop_long_position:
    long_position.remove(ticker)

for ticker in short_position:   
    if ticker not in returns.columns:
        drop_short_position.append(ticker)

for ticker in drop_short_position:
    short_position.remove(ticker)

In [17]:
len(long_position)

2960

In [18]:
len(short_position)

526

In [19]:
# marketcap das ações em posição comprado
marketcap_long_position = list(factors[factors['TAQ_TICKER'].isin(long_position)]['MARKETCAP'].values)

# marketcap das ações em posição vendido
marketcap_short_position = list(factors[factors['TAQ_TICKER'].isin(short_position)]['MARKETCAP'].values)

# soma dos marketcaps das ações em posição comprado
sum_marketcap_long_position = sum(marketcap_long_position)

# soma dos marketcaps das ações em posição vendido
sum_marketcap_short_position = sum(marketcap_short_position)

# value weight das ações em posição comprado
weight_long_position = marketcap_long_position/sum_marketcap_long_position

# value weight das ações em posição vendido
weight_short_position = marketcap_short_position/sum_marketcap_short_position

In [20]:
(returns[long_position]*weight_long_position).sum(axis=1) - (returns[short_position]*weight_short_position).sum(axis=1)

Time
93100    -0.000659
93200    -0.000510
93300     0.000372
93400     0.000209
93500    -0.000520
            ...   
155600    0.000029
155700   -0.000278
155800   -0.000088
155900    0.000058
160000    0.000056
Length: 390, dtype: float64